In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Dropout
from tensorflow.keras.models import Model
from itertools import product

import functions as fs
from functions import make_state, run, get_optimal_action_and_value, get_optimal_actions_and_values, get_model_actions
from functions import test_model, one_batch, one_batch_supervised, train, train_supervised, create_model

## 测试初始值

In [2]:
def init_test(n_tests):
    optimilizer = 'adam'
    n_hidden_layers = 1
    n_dense_units = 512
    ratio_dropout = 0.5
    res = []
    for i in np.arange(n_tests):
        model = create_model(n_hidden_layers, n_dense_units, ratio_dropout, optimilizer)
        scores = np.array([one_batch(model, batch_size=128, n_test_rounds=10000) for i in np.arange(200)])[:,0]
        argmax = scores.argmax()
        res.append((argmax, scores[argmax]))
        print(argmax, scores[argmax])
    return res

In [3]:
res = init_test(100)

79 0.8571709277948568
87 0.8653175709942555
88 0.8637976183042847
78 0.8613568222359931
72 0.8717918856758217
98 0.8573296544630377
81 0.8507270706642879
78 0.8599106853271913
69 0.8597774567895711
88 0.8647716854845486
79 0.8614601356532937
80 0.8752916727911768
87 0.8627165526923445
93 0.8647125677277727
79 0.8701788431992632
79 0.8653295598593406
79 0.8672281308640309
88 0.8726065658550078
82 0.8677729578418586
92 0.8673337271137045
75 0.867652461190452
70 0.8624408737298037
98 0.860786449933764
98 0.8648658947722877
71 0.8560490649114677
69 0.8671878345999346
91 0.8694325160332497
78 0.8648078043010256
80 0.863260696556973
65 0.8636437343788095
103 0.8611513202642295
74 0.8626823481102474
71 0.8686891504689874
88 0.866125431523806
83 0.8673513072684528
112 0.8588906991607484
93 0.8598402416237535
68 0.8633923879610645
71 0.8711164789391416
82 0.8666933062935954
92 0.862650762796969
75 0.8670295557884563
93 0.8632446785615655
81 0.8551121286277951
89 0.8643318422045205
83 0.85634920